<a href="https://colab.research.google.com/github/rlevy820/ds2002/blob/main/Data_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentation

## How To Use
1. If you don't have the libraries installed, run the following code
```
!pip install pandas
!pip install requests
!pip install pysqlite3
```
2. Get API key (click the link and follow instructions
```
https://marketstack.com/
```
- Update apikey in `get_json()`
```
get_json():
    # load in api key
    apikey = <apikey>
```
3. If csv file does not load properly, you can download a copy from the link below
```
https://www.kaggle.com/datasets/valakhorasani/mobile-device-usage-and-user-behavior-dataset
```
4. Under *User Interface* subsection, choose `csv_to_json` or `json_to_csv` and run the cell

## ETL Elements
- `get_json()`
get json file from api request
- `get_csv()`
get csv file from kaggle csv stored in github
- `json_to_csv()`
convert json to csv
- `csv_to_json()`
convert csv to json
- `modify_csv()`
ask user to add/drop columns from the csv
- `modify_json()`
ask user to add/drop columns from the json
- `csv_to_sql()`
convert csv to sql
- `json_to_sql()`
convert json to sql
- `check_db_created()`
print out created db to double check it was created properly
- `json_to_csv_to_sql()`
runs the relevant functions above for the path json -> csv -> sql
- `csv_to_json_to_sql()`
runs the relevant functions above for the path csv -> json -> sql

# Import Libraries

In [161]:
import json
import pandas as pd
import requests
from google.colab import files
import os
from datetime import datetime
import sqlite3

# Define Key Functions

In [162]:
def json_to_csv(json_file, csv_filename):
    # json to df
    data_list = []
    for entry in json_file['data']:
        # add data
        readable_date = datetime.strptime(entry['date'], "%Y-%m-%dT%H:%M:%S%z").strftime("%m/%d/%y %I %p")
        data_list.append([readable_date, entry['symbol'], entry['open'], entry['close'], entry['high'], entry['low'], entry['volume']])

    # create df
    columns = ['date', 'symbol', 'open', 'close', 'high', 'low', 'volume']
    df = pd.DataFrame(data_list, columns=columns)

    # df to csv
    df.to_csv(csv_filename, index=False)
    print(f"Converted JSON to CSV: {csv_filename}")
    return csv_filename

def csv_to_json(df, json_filename):
    # df to json
    json_data = df.to_json(orient='records', date_format='iso')

    # save json
    with open(json_filename, 'w') as f:
        f.write(json_data)

    print(f"Converted CSV to JSON: {json_filename}")
    return json_filename

def get_json():
    # load in api key
    apikey = ''
    # tickers
    tickers = 'tsla'
    # load in url
    url = f'http://api.marketstack.com/v1/eod?access_key={apikey}&symbols={tickers}'
    response = requests.get(url)
    if response.status_code == 200: # if api code works
        # get data as json file
        data = response.json()
        return data
    else:
        return("API Request Error")

def get_csv():
    try:
        data = pd.read_csv('https://raw.githubusercontent.com/rlevy820/ds2002/refs/heads/main/user_behavior_dataset.csv')
        return data
    except FileNotFoundError: # error uploading csv
        print("Error: CSV file not found.")
        return None
    except Exception as e:
        print(f"An error occurred while processing the CSV: {e}")
        return None

def modify_csv(csv_file):
    # load csv into df
    stock_columns = ['date', 'symbol', 'open', 'close', 'high', 'low', 'volume']
    df = csv_file[stock_columns].copy().head()

    # pre-mod summary
    num_records, num_columns = df.shape
    print(f"Number of records (rows): {num_records}")
    print(f"Number of columns: {num_columns}")

    # display df for confirmation
    display(df)

    # validate drop columns for stock data
    def get_valid_drop_columns():
        while True:
            print(f"which columns to drop? available: {', '.join(df.columns)}. if none, type 'none'.")
            to_drop = input()

            if to_drop.lower() == 'none':
                return []

            # process and validate input for dropping columns
            cleaned_input = to_drop.replace(',', ' ').strip()
            cols_to_drop = [col.strip() for col in cleaned_input.split() if col.strip()]
            invalid_cols = [col for col in cols_to_drop if col not in df.columns]

            if invalid_cols:
                print(f"invalid columns: {', '.join(invalid_cols)}. valid options: {', '.join(df.columns)}.")
            else:
                return cols_to_drop

    # validate add columns for stock data
    def get_valid_add_columns():
        valid_add_columns = ['volatility', 'range']

        while True:
            print("which columns to add? options: volatility, range. if none, type 'none'.")
            to_add = input()

            if to_add.lower() == 'none':
                return []

            # process and validate input for adding columns
            cleaned_input = to_add.replace(',', ' ').strip()
            cols_to_add = [col.strip() for col in cleaned_input.split() if col.strip()]
            invalid_adds = [col for col in cols_to_add if col not in valid_add_columns]

            if invalid_adds:
                print(f"invalid columns: {', '.join(invalid_adds)}. valid options: {', '.join(valid_add_columns)}.")
            else:
                return cols_to_add

    # get columns to drop
    cols_to_drop = get_valid_drop_columns()

    # drop columns if necessary
    if cols_to_drop:
        df.drop(columns=cols_to_drop, axis=1, inplace=True)
        print(f"successfully dropped columns: {', '.join(cols_to_drop)}")

    # get columns to add
    cols_to_add = get_valid_add_columns()

    # add columns if necessary
    if 'volatility' in cols_to_add:
        df['volatility'] = df['high'] - df['low']
        print("added 'volatility' column.")

    if 'range' in cols_to_add:
        df['range'] = (df['high'] - df['low']) / df['open'] * 100
        print("added 'range' column.")

    display(df)

    # post-mod summary
    num_records, num_columns = df.shape
    print(f"Number of records (rows): {num_records}")
    print(f"Number of columns: {num_columns}")

    return df

def modify_json(json_file):
    # parse json into df for easier handling
    screen_columns = ['device_model', 'app_time', 'screen_time', 'age', 'gender']
    data_list = []


    for entry in json_file:
        data_list.append([entry['Device Model'], entry['App Usage Time (min/day)'],
                          entry['Screen On Time (hours/day)'], entry['Age'], entry['Gender']])

    df = pd.DataFrame(data_list, columns=screen_columns).head()

    # pre-mod summary
    num_records, num_columns = df.shape
    print(f"Number of records (rows): {num_records}")
    print(f"Number of columns: {num_columns}")

    display(df)

    # validate drop columns for screen time data
    def get_valid_drop_columns():
        while True:
            print(f"which columns to drop? available: {', '.join(df.columns)}. if none, type 'none'.")
            to_drop = input()

            if to_drop.lower() == 'none':
                return []

            # process and validate input for dropping columns
            cleaned_input = to_drop.replace(',', ' ').strip()
            cols_to_drop = [col.strip() for col in cleaned_input.split() if col.strip()]
            invalid_cols = [col for col in cols_to_drop if col not in df.columns]

            if invalid_cols:
                print(f"invalid columns: {', '.join(invalid_cols)}. valid options: {', '.join(df.columns)}.")
            else:
                return cols_to_drop

    # validate add columns for screen time data
    def get_valid_add_columns():
        valid_add_columns = ['time_to_age_ratio', 'app_usage_percent']

        while True:
            print("which columns to add? options: time_to_age_ratio, app_usage_percent. if none, type 'none'.")
            to_add = input()

            if to_add.lower() == 'none':
                return []

            # process and validate input for adding columns
            cleaned_input = to_add.replace(',', ' ').strip()
            cols_to_add = [col.strip() for col in cleaned_input.split() if col.strip()]
            invalid_adds = [col for col in cols_to_add if col not in valid_add_columns]

            if invalid_adds:
                print(f"invalid columns: {', '.join(invalid_adds)}. valid options: {', '.join(valid_add_columns)}.")
            else:
                return cols_to_add

    # get columns to drop
    cols_to_drop = get_valid_drop_columns()

    # drop columns if necessary
    if cols_to_drop:
        df.drop(columns=cols_to_drop, axis=1, inplace=True)
        print(f"successfully dropped columns: {', '.join(cols_to_drop)}")

    # get columns to add
    cols_to_add = get_valid_add_columns()

    # add columns if necessary
    if 'time_to_age_ratio' in cols_to_add:
        df['time_to_age_ratio'] = df['screen_time'] / df['age']
        print("added 'time_to_age_ratio' column.")

    if 'app_usage_percent' in cols_to_add:
        df['app_usage_percent'] = (df['app_time'] / (df['screen_time'] * 60)) * 100
        print("added 'app_usage_percent' column.")

    display(df)

    # post-mod summary
    num_records, num_columns = df.shape
    print(f"Number of records (rows): {num_records}")
    print(f"Number of columns: {num_columns}")

    return df

def csv_to_sql():
  # remove pre-existing db files
  if os.path.exists('etl_storage.db'):
      os.remove('etl_storage.db')

  # create new db
  conn = sqlite3.connect('etl_storage.db')
  modified_df.to_sql('modified_csv_table', conn, if_exists='replace', index=False)

  conn.commit()
  conn.close()

def json_to_sql():
  # remove pre-existing db files
  if os.path.exists('etl_storage.db'):
      os.remove('etl_storage.db')

  # create new db
  conn = sqlite3.connect('etl_storage.db')
  modified_json_df.to_sql('modified_csv_table', conn, if_exists='replace', index=False)

  # close db connection
  conn.commit()
  conn.close()

def check_db_created():
  # connect to db
  conn = sqlite3.connect('etl_storage.db')
  cursor = conn.cursor()

  # get table
  cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
  tables = cursor.fetchall()

  print("Tables in the database:")
  for table in tables:
      print(table[0])

  # select all from table
  cursor.execute("SELECT * FROM modified_csv_table;")
  rows = cursor.fetchall()

  # print db
  print("\nData in modified_csv_table:")
  for row in rows:
    print(row)

  conn.close()

def json_to_csv_to_sql():
  json_data = get_json()
  csv_file = json_to_csv(json_data, 'stock_data.csv') if isinstance(json_data, dict) else print(json_data)  # convert to json
  modified_df = modify_csv(pd.read_csv(csv_file))
  csv_to_sql()
  check_db_created()

def csv_to_json_to_sql():
  csv_data = get_csv()
  csv_to_json(csv_data, 'csv_data.json')
  with open('csv_data.json', 'r') as json_file:
    json_data = json.load(json_file)
  modified_json_df = modify_json(json_data)
  json_to_sql()
  check_db_created()

## User Interface

In [164]:
choose_converter = "csv to json" #@param ["csv to json", "json to csv"]

if choose_converter == "csv to json":
  csv_to_json_to_sql()
elif choose_converter == "json to csv":
  json_to_csv_to_sql()


Converted CSV to JSON: csv_data.json
Number of records (rows): 5
Number of columns: 5


,device_model,app_time,screen_time,age,gender
0,Google Pixel 5,393,6.4,40,Male
1,OnePlus 9,268,4.7,47,Female
2,Xiaomi Mi 11,154,4.0,42,Male
3,Google Pixel 5,239,4.8,20,Male
4,iPhone 12,187,4.3,31,Female


which columns to drop? available: device_model, app_time, screen_time, age, gender. if none, type 'none'.

which columns to add? options: time_to_age_ratio, app_usage_percent. if none, type 'none'.



,device_model,app_time,screen_time,age,gender
0,Google Pixel 5,393,6.4,40,Male
1,OnePlus 9,268,4.7,47,Female
2,Xiaomi Mi 11,154,4.0,42,Male
3,Google Pixel 5,239,4.8,20,Male
4,iPhone 12,187,4.3,31,Female


Number of records (rows): 5
Number of columns: 5
Tables in the database:
modified_csv_table

Data in modified_csv_table:
('Google Pixel 5', 393, 6.4, 40, 'Male')
('OnePlus 9', 268, 4.7, 47, 'Female')
('Xiaomi Mi 11', 154, 4.0, 42, 'Male')
('Google Pixel 5', 239, 4.8, 20, 'Male')
('iPhone 12', 187, 4.3, 31, 'Female')
